# Bienvenido a tu primer competencia de ciencia de datos!!!

En este notebook desarrollaremos el problema del Titanic. Este dataset es el más popular entre todos los que quieren iniciar en este mundo de la Ciencia de Datos. Básicamente se busca predecir si un pasajero, con ciertas caractéristicas, sobrevivió o no a esta trEdaddia ocurrida en el siglo 20. 

Recuerda que para comenzar a hacer el análisis,  tenemos que involucrarnos con el problema en sí y conocer nuestra data. Así que comencemos! 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "Pega tu dirección de Drive acá"

In [ ]:
import pandas as pd  #Sirve para manejar y manipular la data en forma de dataframe
import numpy as np #Calculos vectoriales que pueden ser aplicados al dataframe
import matplotlib.pyplot as plt #Para darle estructura y desplegar el gráfico 
import seaborn as sns #Para hacer el gráfico todo bonito y chulo 
import random #Una librería para randomizar 

#En este notebook usaremos:

import missingno as ms #Librería muy útil para ver los valores nulos

#Usualmente salen advertencias de actualizaciones pasadas de las librerías, las ignoramos por ahora...
import warnings
warnings.filterwarnings('ignore')

En este caso, tenemos dos tablas de datos: train y summit (test.csv). Ahora, esta data ha sido sacada de una **competencia de Kaggle** Por eso, la data que nombraremos como **summit contendrá la base para poder aplicar nuestro modelo ya completado**. El mismo Kaggle se encargará de saber si es un buen modelo o no. Posteriormente, esta tabla nombrada **train nos servirá tanto para entrenar como para testear nuestro modelos**. Así que no te sorprendas si es que más adelante dividimos este train en dos dataframes diferentes...

El link de la competencia es: https://www.kaggle.com/c/titanic

In [ ]:
train = pd.read_csv('train.csv')

summit = pd.read_csv('test.csv')

In [ ]:
#head

In [ ]:
#Jugaremos y mostraremos a lo largo de la clase conceptos varios...

**Explicaremos las variables que están dentro de la data**

**Pclass**: Status socioeconómico 1st = ClaseAlta 2nd=ClaseMedia 3rd=ClaseBaja 

**Hermanos**: El dataset define relaciones familiares, es decir, nos dice el número de hermanos.

**Padres_hijos**: Número de padres / hijos dentro del Titanic

**Esposo_esposa** = marido / Mujer (se obvia novias o enamoradas)

**Tarifa**: El costo del ticket que se compró para el pasaje

**Sobrevivio**: Si la persona sobrevivió o no.	0 = No, 1 = Sí

**Ticket***: La clase de ticket que sacó el pasajero.

**P_embarque**: Puerta de embarcación,	C = Cherbourg, Q = Queenstown, S = Southampton

**Cabin**: El número de cabina

In [ ]:
print('Para la data en train, tendremos:')
print('--------------------------------------------------------------------------------------------')
for n in train.columns:
    print('El número de valores diferentes en la columna', n, 'es:', len(train[n].unique()))
print('--------------------------------------------------------------------------------------------')
#Ahora buscaremos el número de valores nulos dentro de la data
for n in train.columns:
    if train[n].isnull().values.any() == True:
        print('Existen' , train[n].isna().sum(), 'valores nulos en la columna', n)

In [ ]:
print('Para la data que summitearemos, tendremos:')
print('-------------------------------------------------------------------------------')
for n in summit.columns:
    print('El número de valores diferentes en la columna', n, 'es:', len(summit[n].unique()))
print('-------------------------------------------------------------------------------')
#Ahora buscaremos el número de valores nulos dentro de la data
for n in summit.columns:
    if summit[n].isnull().values.any() == True:
        print('Existen' , summit[n].isna().sum(), 'valores nulos en la columna', n)

## Missingno: Una forma más interactiva de ver mis nulos.

Esta librería nos grafica nuestro dataframe de tal manera en la que vemos qué columnas tienen datos vacíos. Esto es representado por los espacios en blanco que tenemos. Es más intuitivo y fácil de mapear cuáles son los problemas de los nulos dentro de nuestro dataset. 

In [ ]:
ms.matrix(train)

In [ ]:
ms.matrix(summit)

**Eliminaremos** los valores que nos parecen **irrelevantes** para predecir si una persona sobrevive o no dentro del Titanic. En este caso, intuimos que son:

- Nombre: ¿Qué tiene que ver el nombre con que se sobreviva o no?


- Ticket: Si bien se podría pensar que el ticket nos daría algún criterio, en este caso no se ha encontrado una relación directa. Ah pero eso no quita que la podamos usar. Puedes ponerte creativo y sacar una conclusión!


- PassengerId: Lo mismo que la variable Ticket. Asumimos que no tiene relación con la supervivencia y lo quitamos.

In [ ]:
train.drop(['Name','Ticket','IdPasajero','Cabin'], axis = 1,inplace=True)
summit = summit.drop(['Name','Ticket','IdPasajero','Cabin'], axis = 1)

#### Explicaremos brevemente cómo filtrar datos dentro de un dataframe (mask method) 

In [ ]:
#Máscaras generales

In [ ]:
#Método loc: Cómo hacer una máscara y agarrar la columna que queramos

## Reemplazando Nulos: 

Un valor Nulo no es más que data que no ha sido recogida (o imputada), osea **data vacía**. Esta nulidad puede deberse a diferentes casos y las maneras en las que podemos lidiar con ellos son varias. La forma en cómo podemos tratar con este tipo de variable depende de diferentes factores como la cantidad, la dispersión y la importancia de la variable para el target (recuerda que target es la variable que vamos a predecir). 



Ahora llenaremos los valores NaN de las columnas Edad y P_embarque. En la columna Edad llenaremos de manera random definiendo un intervalo en el que nuestra data va iterar. Ese intervalo será definido por **(media +- DesviaciónEstandar)**

In [ ]:
#media y std

In [ ]:
print("La edad media de nuestro train dataset es:",train['Edad'].mean(),"\nLa desviación estandar de la edad de nuestro train dataset es:",train['Edad'].std())
print("La edad media de nuestro summitdataset es:",summit['Edad'].mean(),"\nLa desviación estándar de la edad de nuestro summites:",summit['Edad'].std())

In [ ]:
#Quantiles y moda(con plot)

##### Crearemos una función para llenarlo automáticamente:

Recuerda que una función es...

. Un conjunto lógico de código que se une para hacerlo generalizado.


. Una función tiene parámetros necesarios al llamarlo y a su vez parámetros por default que pueden ser alterados.



### Punto de decisión!!!

Acá vas a poder decidir entre dos métodos!! 

- Llenar los nulos con la media directamente en una línea de código.

- LLenar los nulos randomizando por sobre un intervalo definido por la media y la desv.estándar. 

In [ ]:
#SI QUIERES LLENARLO DE LA PRIMERA MANERA, CORRE ESTE CODIGO:

train['Edad'].fillna(int(train['Edad'].mean()),inplace=True)
summit['Edad'].fillna(int(train['Edad'].mean()),inplace=True)

In [ ]:
#SI QUIERES LLENARLO DE LA SEGUNDA MANERA, CORRE ESTE CODIGO. Pero desbloquealo primero:

#Crearemos una función para llenar de manera random dentro del intervalo de la media y desv stand (std)
def llena_nan_con_media_y_std(df,col='Edad'):
    nan = df[df[col].isna()] #el dataset de los que tienen la columna (col = 'Edad' ) con valores nulos
    min_ = df[col].mean() - df[col].std() #El comienzo de nuestro intervalo, la media menos la desv. estandar.
    max_ = df[col].mean() + df[col].std() #El final de nuestro intervalo, la media más la desv. estandar.
    for i in nan.index: #Para cada i en los index de mi data nula
        random_num = random.uniform(int(min_),int(max_)) #randomizo dentro de los intervalos que quiero y especifico que quiero enteros
        df[col].loc[i] = random_num #Reemplazo los valores dentro de mi data original.

In [ ]:
llena_nan_con_media_y_std(train) #Aplico la función para train
llena_nan_con_media_y_std(summit) #Aplico la función para summit

Ahora tenemos que hacernos cargo de los valores nulos de la columna P_embarque y Tarifa dentro de los dos dataframes (summit y train)

A pesar de que podamos llenarlo con la moda directamente con fillna, prefiero ver con mis propios ojos qué tanto difiere la moda de su segundo valor más recurrente....

In [ ]:
# Visualizaremos los datos de la columna P_embarque en nuestra data de entrenamiento. #countplot:


In [ ]:
# Ahora sí, podemos llenar los nulos de P_embarque de esta manera:
train['P_embarque'].fillna(train['P_embarque'].mode(), inplace=True)
summit['P_embarque'].fillna(train['P_embarque'].mode(), inplace=True)

In [ ]:
#Ahora lo hacemos para el summit data (solamente el Tarifa, 1 dato nulo)
summit['Tarifa'].fillna(summit['Tarifa'].mean(),inplace=True)

In [ ]:
# Seguiremos ploteando para ver qué ha pasado con los nulos (missigno)

## Detectando y tratando outliers

- Uno de los mejores gráficos para poder ver los outliers es el **grafico de caja** o **boxplot** que nos grafica los cuantiles de la data y nos dice qué tan pronunciados son nuestros outliers con respecto a ellos.

In [ ]:
plt.style.use('ggplot') #Es el estilo que le ponemos a la data
plt.figure(figsize=(4,6))
sns.boxplot(train['Edad'], orient="v") #El gráfico de distribución
plt.show()
plt.figure(figsize=(4,6))
sns.boxplot(train['Tarifa'], orient='v')
plt.show()

Viendo el boxplot, con la variable Edad no me haré tanto problema. Sino lo que me interesa reemplazar son los outliers de la variable 
Reemplazaremos los outliers de Tarifa por los valores que tengamos en el 95% de los datos:

In [ ]:
#Plotea la distribución de Tarifa

In [ ]:
#Reemplazando los outliers con los cuantiles 95 y etc

outliers_a_reemplazar = train[train['Tarifa'] > train['Tarifa'].quantile(.95)].index
train.loc[outliers_a_reemplazar,'Tarifa'] = train['Tarifa'].quantile(.95)

In [ ]:
outliers_a_reemplazar = summit[summit['Tarifa']> summit['Tarifa'].quantile(.95)].index
summit.loc[outliers_a_reemplazar,'Tarifa'] = summit['Tarifa'].quantile(.95)

In [ ]:
#Ploteemos denuevo Tarifa y Edad... Para ver un antes y un después

#### Advertencia!!!  No siempre es la mejor opción... Podríamos crear otra variable para indicar que estamos reemplazando los datos._

## Podemos (y debemos) crear otros features a la data:

Hemos visto que hay dos variables que se relacionan a la familia del pasajero (Hermanos & Padres_hijos), asi que por qué no creamos una variable que resuma la cantidad de familiares de una persona? Y también contar si la persona viaja sola o acompañada en la nave.

In [ ]:
#Creamos la variable miembros de familia. 

train['Miembros_de_fam'] = train['Hermanos'] + train['Padres_hijos'] + 1 #(1 reempresenta a él o ella)
summit['Miembros_de_fam'] = summit['Hermanos'] + summit['Padres_hijos'] + 1

#Crearemos la variable Alone para nuestra data
train['Viaja_solo'] = 1 
train['Viaja_solo'].loc[train['Miembros_de_fam'] > 1] = 0

summit['Viaja_solo'] = 1 
summit['Viaja_solo'].loc[summit['Miembros_de_fam'] > 1] = 1

In [ ]:
train.head()

#### Dándole sentido a nuestra variable... 

**Pensando el problema**: ¿Qué es lo que pasa en un desastre natural? No le dan prioridad a las mujeres y niños para que puedan ir directo a las naves de rescate? Entonces, sería conveniente identificar si el pasajero es Hombre, Mujer o **niño** Para esto, crearemos la variable niño. 

Pero **vas a tener que decidir** porque puedes:

- Crear una nueva categoría para identificar los niños dentro de la columna género

- Crear una columna extra donde nos diga si es niño = 1, si no es niño = 0

In [ ]:
#RETO: Crear una variable dummy de niño o crear una variable en la que reemplaces la variable categórica dentro del género.

In [ ]:
#Solucion 1
train.loc[(train['Edad'] < 15),'Genero'] = 'niño'
summit.loc[(summit['Edad']<15),'Genero'] = 'niño'
#Solucion2
train['es_niño'] = 0
train.loc[(train['Edad'])<15,'es_niño'] = 1

summit['es_niño'] = 0
summit.loc[(summit['Edad']<15),'es_niño'] = 1

## Data Visualization:

**Por qué es importante visualizar nuestros datos?**:

In [ ]:
plt.style.use('ggplot') #Para ponerle el estilo tipo R
plt.figure(figsize=(8,6)) #Podemos modificar la imagen 
plt.title('Vemos la data filtrada por supervivencia, año y género') #Le ponemos título a la imagen
sns.swarmplot(x = 'Sobrevivio',y='Edad', 
              data=train, linewidth=1,hue='Genero', palette = 'muted')

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Vemos el % de supervivencias según puerta de embarcación (P_embarque)')
sns.countplot(x = 'P_embarque', data = train[train['Sobrevivio'] == 1])

Con lo gráficos anteriores, ya tenemos una intuición acerca de **cómo deberíamos reemplazar los datos**, es decir, ahora podemos hacer el encoding de manera intuitiva.

In [ ]:
#Ahora podemos decir qué prioridades de categorías hay dentro de las variables P_embarque y Genero
cat_to_nums = {"P_embarque":  {"S": 2, "C": 1, "Q":0},
               "Genero": {"male":0,"niño":1,"female":2}}
#Vamos a reemplazarlo en el train y en el summit

train.replace(cat_to_nums, inplace = True)
summit.replace(cat_to_nums, inplace = True)

In [ ]:
train.head()

## Normalizando mi data:
Cuándo normalizar o standarizar?: https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc

Veamos la distribución de nuestros dos datos netamente numéricos que tenemos:


In [ ]:
sns.distplot(train['Tarifa']) #El gráfico de distribución
plt.show()
sns.distplot(train['Edad'])
plt.show()

In [ ]:
from sklearn.preprocessing import normalize #Para la normalización de la data.

In [ ]:
# Normalizando 'Tarifa' & 'Edad' para summitear dataset y train
X_to_norm = [train['Tarifa'],
     train['Edad']] #Asigno las variables que quiero normalizar

X_normalize = normalize(X_to_norm) #Le aplico la normalización

X_2_to_norm = [summit['Tarifa'],
      summit['Edad']]  #El mismo procedimiento para el summir

X_2_normalize = normalize(X_2_to_norm)

#Asigno la variable normalizada a mi df

train = train.assign(Tarifa = X_normalize[0]) 

train = train.assign(Edad = X_normalize[1])

summit= summit.assign(Tarifa = X_2_normalize[0])

summit= summit.assign(Edad = X_2_normalize[1])

In [ ]:
#Now we have all numerical values!
train.head()

In [ ]:
plt.style.use('ggplot') #Es el estilo que le ponemos a la data 
sns.distplot(train['Tarifa']) 
plt.show()
sns.distplot(train['Edad'])
plt.show()

## Guardamos nuestro trabajo hasta ahora

In [ ]:
import dill

In [ ]:
dill.dump_session('Titanic01.db')